In [1]:
import glob
files = sorted(glob.glob("*.csv"))
display(len(files))

319

In [2]:
import numpy as np
import pandas as pd
#epochs = np.array([]).reshape(0,136)

epochs = list()

for epoch in files:
  #print(epoch)
  dfAux = pd.read_csv(epoch,sep=',', index_col=False)
  del dfAux['TTIME']
  for col in dfAux.columns:
    dfAux[col] = dfAux[col].astype('float64')
  #epochs = np.vstack((epochs,dfAux.values))
  epochs.append(dfAux.values)
  dfAux = None

epochsArray = np.array(epochs)
display(epochsArray.shape)
display(epochsArray[0])

C:\Users\Bryan\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


(319,)

array([[247., 212., 249., ..., 283., 299., 282.],
       [247., 214., 248., ..., 281., 296., 280.],
       [245., 216., 246., ..., 281., 293., 280.],
       ...,
       [227., 194., 229., ..., 278., 309., 277.],
       [228., 192., 229., ..., 274., 307., 273.],
       [227., 191., 229., ..., 275., 306., 273.]])

In [3]:
#Lableing the data
#0 = Lie / 1 = Truth

labels = np.array([])

for i in range(320):
  if i % 2 == 0:
    labels = np.append(labels,1)
  else:
    labels = np.append(labels,0)

#Removing the Label of video WF010_4PL since it has incomplete data
labels = np.delete(labels,187)

display(labels.shape)

#Lableing the data
#0 = Lie / 1 = Truth

(319,)

In [4]:
from sklearn.preprocessing import MinMaxScaler
def to_sequences(seq_size, obs, label):
    x = []
    y = []
    scaler = MinMaxScaler()
    obs = scaler.fit_transform(obs)
    for i in range(len(obs)-seq_size):
        window = obs[i:(i+seq_size)]
        x.append(window)
        y.append(label)
        
    return np.array(x),np.array(y)
    
SEQUENCE_SIZE = 10

xAux0, yAux0 = to_sequences(SEQUENCE_SIZE,epochsArray[0],labels[0])
xAux1, yAux1 = to_sequences(SEQUENCE_SIZE,epochsArray[1],labels[1])

X = np.vstack((xAux0,xAux1))
y = np.hstack((yAux0,yAux1))

In [5]:
for i in range(317): #82
  xAux, yAux = to_sequences(SEQUENCE_SIZE, epochsArray[i+2], labels[i+2])
  X = np.vstack((X,xAux))
  y = np.hstack((y,yAux))

print(X.shape)
print(y.shape)

(226729, 10, 136)
(226729,)


In [6]:
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LayerNormalization 
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
dummy_y = to_categorical(encoded_Y)

In [7]:
from tensorflow.keras.models import load_model
modeloLeido = load_model("modelo8_stacked_GRU.h5")

In [12]:
def percentage(part, whole):
  return 100 * float(part)/float(whole)

def resultado(truth, lie):
  if truth > lie:
    return (truth, "Truth")
  elif lie > truth:
    return (lie, "Deception")
  else:
    return (truth, "Uncertain")

AF_LIST = []
PF_LIST = []
RESULTS = []

for i in range(319): #84
    xAuxAux0, yAuxAux0 = to_sequences(SEQUENCE_SIZE,epochsArray[i],labels[i])
    y_prob = modeloLeido.predict(xAuxAux0)
    y_classes = y_prob.argmax(axis=-1)
        
    truth_1 = np.count_nonzero(y_classes)
    size = y_classes.size
    lie_0 = size - truth_1
    
    result2 = np.mean(y_prob, axis = 0)#AVERAGE FIDELITY FROM VIDEO i(lie,truth)
    
    AF_TRUTH = result2[1] * 100.0
    AF_LIE = result2[0] * 100.0
    AF = [AF_LIE, AF_TRUTH]
    AF_LIST.append(AF)
    
    PF_TRUTH = percentage(truth_1, size)
    PF_LIE = percentage(lie_0,size)
    PF = [PF_LIE, PF_TRUTH]#PROPORTIONAL FIDELITY FROM VIDEO i(lie,truth)
    PF_LIST.append(PF)
    
    res = resultado(truth_1, lie_0)
    result = res[1] #String de "Verdad" o "Mentira"
    RESULTS.append(result)
    


In [13]:
display(AF_LIST)

[[6.339962035417557, 93.66005063056946],
 [97.24839925765991, 2.7515834197402],
 [12.22158670425415, 87.7784013748169],
 [86.32359504699707, 13.676509261131287],
 [0.687694177031517, 99.31229948997498],
 [96.22048735618591, 3.7795238196849823],
 [1.089081633836031, 98.91103506088257],
 [99.43532347679138, 0.5646876990795135],
 [6.260005384683609, 93.73998641967773],
 [94.70447897911072, 5.2955977618694305],
 [15.196336805820465, 84.8036527633667],
 [92.60258674621582, 7.397367060184479],
 [2.0127059891819954, 97.98728823661804],
 [92.74433255195618, 7.255679368972778],
 [2.3227211087942123, 97.67728447914124],
 [94.5766270160675, 5.423429235816002],
 [5.068845301866531, 94.93110775947571],
 [95.11345624923706, 4.886629804968834],
 [13.81770372390747, 86.18229627609253],
 [85.49156785011292, 14.508403837680817],
 [2.511298283934593, 97.48877882957458],
 [90.3559923171997, 9.644008427858353],
 [4.23920676112175, 95.76079249382019],
 [98.69372844696045, 1.3062852434813976],
 [2.9349410906

In [14]:
display(PF_LIST)

[[4.133333333333334, 95.86666666666666],
 [99.74025974025975, 0.2597402597402597],
 [8.181818181818182, 91.81818181818181],
 [91.64556962025317, 8.354430379746836],
 [0.136986301369863, 99.86301369863014],
 [98.13559322033899, 1.8644067796610169],
 [0.410958904109589, 99.58904109589041],
 [100.0, 0.0],
 [2.3728813559322033, 97.62711864406779],
 [96.19718309859155, 3.8028169014084505],
 [10.0, 90.0],
 [94.8, 5.2],
 [1.320754716981132, 98.67924528301887],
 [96.36363636363636, 3.6363636363636362],
 [0.13333333333333333, 99.86666666666666],
 [97.06666666666666, 2.933333333333333],
 [1.746031746031746, 98.25396825396825],
 [97.04225352112677, 2.9577464788732395],
 [8.76923076923077, 91.23076923076923],
 [90.63492063492063, 9.365079365079366],
 [1.3333333333333333, 98.66666666666667],
 [96.0, 4.0],
 [2.2535211267605635, 97.74647887323944],
 [99.87012987012987, 0.12987012987012986],
 [3.018867924528302, 96.98113207547169],
 [98.40579710144928, 1.5942028985507246],
 [8.904109589041095, 91.0958

# End of AF and PF results for the Stacked GRU (best model)